<a href="https://colab.research.google.com/github/rhmes/point-cloud-compression/blob/main/point_cloud_compression_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Point Cloud Compression Demo

This notebook demonstrates installation, core functionality, and testing for the point cloud compression project.

In [1]:
# Connect to Google Drive (for Colab users)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Google Drive mounted.')
except ImportError:
    print('Not running in Colab, skipping Google Drive mount.')

Mounted at /content/drive
Google Drive mounted.


In [2]:
# Clone repo only if not already cloned
import os
repo_url = 'https://github.com/rhmes/point-cloud-compression.git'
parent_dir = '/MyDrive/projects'
repo_dir = os.path.join(parent_dir, 'point-cloud-compression')
os.makedirs(parent_dir, exist_ok=True)
os.chdir(parent_dir)
if not os.path.exists('point-cloud-compression'):
    os.system(f'git clone {repo_url}')
else:
    os.system(f'cd {repo_dir} && git pull')
os.chdir(repo_dir)
print(f'Current working directory: {os.getcwd()}')

Current working directory: /MyDrive/projects/point-cloud-compression


In [ ]:
# Set this variable to True to install GPU dependencies, False for CPU
use_gpu = True # @param {type:"boolean"}

# Install dependencies based on the use_gpu flag

import os

if use_gpu:
    print("Setting up GPU virtual environment...")
    venv_dir = 'venv_gpu'
    setup_script = 'venv_gpu_setup.sh'
else:
    print("Setting up CPU virtual environment...")
    venv_dir = 'venv_cpu'
    setup_script = 'venv_cpu_setup.sh'

# Check if the virtual environment directory already exists
if not os.path.exists(venv_dir):
    print(f'No virtual environment found at {venv_dir}. Running setup script...')
    # Run the setup script
    # Using !bash is generally safer in Colab than subprocess for simple script execution
    !bash {setup_script}
else:
    print(f'Virtual environment already exists at {venv_dir}. Skipping setup script.')


# After setting up the environment (or if it already existed), activate and install remaining packages if needed
# Note: The setup scripts might already install most requirements.
# These additional installs are included in case they are not handled by the scripts.

print("Installing additional dependencies...")

# The setup scripts should handle activating the environment internally.
# If not, you would need to explicitly activate it here, but that's tricky with ! commands.
# We'll rely on the setup script doing the activation or the subsequent commands working without explicit activation in this cell.

# Install Open3D (usually the same for CPU and GPU)
# This might be redundant if the setup script already installs it
!pip install open3d

# Install PyTorch3D from source (usually the same for CPU and GPU)
# This might be redundant if the setup script already installs it
!pip install "git+https://github.com/facebookresearch/pytorch3d.git"

print("Dependency installation complete.")

Setting up GPU virtual environment...
No virtual environment found at venv_gpu. Running setup script...
Error: Command '['/MyDrive/projects/point-cloud-compression/venv_gpu/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
Installing additional dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstal

In [ ]:
# Download ModelNet40 and ShapeNet datasets from Google Drive (Option 1 from README)
import os
import urllib.request

def download_from_gdrive(url, output_path):
    try:
        import gdown
    except ImportError:
        os.system('pip install gdown')
        import gdown
    gdown.download(url, output_path, quiet=False)

# ModelNet40
modelnet_url = 'https://drive.google.com/uc?id=1Isa8seckZ9oNzstlE7VZcd6wVVx8LdMF'
modelnet_zip = 'ModelNet40_pc_8192.zip'
if not os.path.exists('data/ModelNet40_pc_8192'):
    print('Downloading ModelNet40 pre-converted point clouds...')
    download_from_gdrive(modelnet_url, modelnet_zip)
    import zipfile
    with zipfile.ZipFile(modelnet_zip, 'r') as zip_ref:
        zip_ref.extractall('data')
    os.remove(modelnet_zip)
    print('ModelNet40 download and extraction complete.')
else:
    print('ModelNet40 dataset already exists. Skipping download.')

# ShapeNet
shapenet_url = 'https://drive.google.com/uc?id=1OzaU01kolBpfRRD0zKESYh67Hh2s2dbD'
shapenet_zip = 'ShapeNet_pc_2048.zip'
if not os.path.exists('data/ShapeNet_pc_2048'):
    print('Downloading ShapeNet pre-converted point clouds...')
    download_from_gdrive(shapenet_url, shapenet_zip)
    import zipfile
    with zipfile.ZipFile(shapenet_zip, 'r') as zip_ref:
        zip_ref.extractall('data')
    os.remove(shapenet_zip)
    print('ShapeNet download and extraction complete.')
else:
    print('ShapeNet dataset already exists. Skipping download.')

In [ ]:
# Example core functionality: List files in the data directory

def list_data_files(data_dir='data'):
    if not os.path.exists(data_dir):
        print(f"Directory '{data_dir}' does not exist.")
        return []
    files = os.listdir(data_dir)
    print(f"Files in '{data_dir}':", files)
    return files

# Run the function
data_files = list_data_files()

In [ ]:
# Compress point cloud test files using the trained model
!python compress.py './data/ModelNet40_pc_01_8192p/**/test/*.ply' './data/ModelNet40_K256_compressed' './model/K256' --K 256

In [ ]:
# Decompress the compressed point cloud files
!python decompress.py './data/ModelNet40_K256_compressed' './data/ModelNet40_K256_decompressed' './model/K256' --K 256

In [ ]:
# Evaluate the compression results using PSNR, Chamfer distance, and bpp metrics
!python eval.py './data/ModelNet40_pc_01_8192p/**/test/*.ply' './data/ModelNet40_K256_compressed' './data/ModelNet40_K256_decompressed' './eval/ModelNet40_K256.csv' '../geo_dist/build/pc_error'

In [ ]:
# Visualize evaluation metrics and save plots
!python visualize.py --csv './eval/ModelNet40_K256.csv' --outdir './figure/'